In [1]:
# SELF REMINDER. Copy the 'ocpa' directory to the forked one from github, so that I can push updates to github.

# Python native
import pickle
from statistics import median as median
from tqdm import tqdm
import os
os.chdir("/home/tim/Development/OCELFeatureExtractionExperiments/")
from copy import deepcopy

# Object centric process mining
from ocpa.algo.predictive_monitoring.obj import Feature_Storage as FeatureStorage

# Data handling
import pandas as pd
import numpy as np

# # Simple machine learning models, procedure tools, and evaluation metrics
# from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PowerTransformer

# PyG
import torch
from replicating.ocpa_PyG_integration.EventGraphDataset import EventGraphDataset

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

# Global variables
from replicating.experiment_config import STORAGE_PATH, RANDOM_SEED, TARGET_LABEL

Torch version: 1.13.1+cu117
Cuda available: True
Torch geometric version: 2.2.0


In [4]:
with open(
    f"{STORAGE_PATH}/raw/BPI2017-feature_storage-[C2,D1,P2,P3,O3].pkl", "rb"
) as file:
    feature_storage: FeatureStorage = pickle.load(file)


feature_storage.extract_normalized_train_test_split(
    test_size=0.3,
    validation_size=0.2,
    scaler=StandardScaler,
    scaling_exempt_features=[],
    state=RANDOM_SEED,
)

with open(f"{STORAGE_PATH}/raw/BPI2017-adams.fs", "wb",) as file:
    pickle.dump(feature_storage, file)

In [2]:
ds_train = EventGraphDataset(
    root=STORAGE_PATH,
    filename="BPI2017-adams.fs",
    label_key=TARGET_LABEL,
    train=True,
    verbosity=51
)
ds_val = EventGraphDataset(
    root=STORAGE_PATH,
    filename="BPI2017-adams.fs",
    label_key=TARGET_LABEL,
    validation=True,
    verbosity=51
)
ds_test = EventGraphDataset(
    root=STORAGE_PATH,
    filename="BPI2017-adams.fs",
    label_key=TARGET_LABEL,
    test=True,
    verbosity=51
)

Processing...
15754it [00:04, 3284.89it/s]
Done!
Processing...
6302it [00:01, 3252.20it/s]
Done!
Processing...
9453it [00:02, 3248.10it/s]
Done!


In [3]:
print("Train set")
print(ds_train.get_summary())
print()

# # Validation set gives error for now
print("Validation set")
print(ds_val.get_summary())
print()

print("Test set")
print(ds_test.get_summary())
print()

Train set


100%|██████████| 15754/15754 [00:03<00:00, 4008.98it/s]


EventGraphDataset (#graphs=15754):
+------------+----------+----------+
|            |   #nodes |   #edges |
|------------+----------+----------|
| mean       |     12.5 |     13.7 |
| std        |      3.6 |      4.5 |
| min        |      6   |      5   |
| quantile25 |     10   |     11   |
| median     |     12   |     13   |
| quantile75 |     14   |     16   |
| max        |     41   |     54   |
+------------+----------+----------+

Validation set
EventGraphDataset (#graphs=6302):
+------------+----------+----------+
|            |   #nodes |   #edges |
|------------+----------+----------|
| mean       |     12.5 |     13.7 |
| std        |      3.6 |      4.5 |
| min        |      6   |      5   |
| quantile25 |     10   |     11   |
| median     |     12   |     13   |
| quantile75 |     14   |     16   |
| max        |     41   |     51   |
+------------+----------+----------+

Test set
EventGraphDataset (#graphs=9453):
+------------+----------+----------+
|            |   #no

In [6]:
from replicating.model import GCN, GAT
import torch
from torch_geometric.loader import DataLoader
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


def count_parameters(model) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


# Initialize model
model = GCN(
    ds_train.num_node_features, {"num_hidden_features": ds_train.num_node_features}
)
print(model)
print(f"Number of parameters: {count_parameters(model)}")


# Use GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
model = model.to(device)
# data = ds_train.to(device)

# Initialize Optimizer
learning_rate = 0.01
# decay = 5e-4
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate,
    #  weight_decay=decay
)
NUM_GRAPHS_PER_BATCH = 64

mse = torch.nn.MSELoss()
mae = torch.nn.L1Loss()
loss_fn = mae

train_loader = DataLoader(ds_train, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)
val_loader = DataLoader(ds_val, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)
test_loader = DataLoader(ds_test, batch_size=128, shuffle=True)


def calculate_metrics(y_pred, y_true, epoch, type):
    print(f"\n MAE: \n {mean_absolute_error(y_pred, y_true)}")
    print(f"MSE: {mean_squared_error(y_true, y_pred)}")
    print(f"R^2: {r2_score(y_true, y_pred)}")


GCN(
  (gconv1): GCNConv(24, 24)
  (gconv2): GCNConv(24, 24)
  (out): Linear(in_features=24, out_features=1, bias=True)
)
Number of parameters: 1225
Device: cuda:0


In [ ]:
def train_one_epoch(epoch_index:int, model, train_loader, optimizer, loss_fn, tb_writer):
    # Enumerate over the data
    running_loss = 0.0
    last_loss = 0
    for i, batch in enumerate(tqdm(train_loader)):
        # Use GPU
        batch.to(device)
        # Every data instance is an input + label pair
        inputs, adjacency_matrix, labels = batch.x.float(), batch.edge_index, batch.y.float()
        # Reset gradients
        optimizer.zero_grad() 
        # Passing the node features and the connection info
        outputs = model(inputs, adjacency_matrix) 
        # Compute loss and gradients
        loss = loss_fn(torch.squeeze(outputs), labels)
        loss.backward()
        # Adjust learnable weights
        optimizer.step()
        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print(f'  batch {i + 1} loss: {last_loss}')
            tb_x = epoch_index * len(train_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.
    
    return last_loss

def run_training(num_epochs, model, train_loader, validation_loader, optimizer, loss_fn, timestamp):
    model_path = f"models/{model.get_class_name()}_{timestamp}"
    if not os.path.exists(model_path):
        os.makedirs(model_path)
    writer = SummaryWriter(f"{model_path}/run")
    best_vloss = 1_000_000_000_000_000.

    for epoch in range(num_epochs):
        print(f'EPOCH {epoch + 1}:')

        # Make sure gradient tracking is on, and do a pass over the data
        model.train(True)
        avg_loss = train_one_epoch(
            epoch, model, train_loader, optimizer, loss_fn, writer
        )

        # We don't need gradients on to do reporting
        model.train(False)

        running_vloss = 0.0
        for i, vdata in enumerate(validation_loader):
            vdata.to(device)
            vinputs, vadjacency_matrix, vlabels = vdata.x.float(), vdata.edge_index, vdata.y.float()
            voutputs = model(vinputs,vadjacency_matrix)
            vloss = loss_fn(voutputs, vlabels)
            running_vloss += vloss

        avg_vloss = running_vloss / (i + 1)
        print(f'LOSS train {avg_loss} valid {avg_vloss}')

        # Log the running loss averaged per batch
        # for both training and validation
        writer.add_scalars('Training vs. Validation Loss',
                        { 'Training' : avg_loss, 'Validation' : avg_vloss },
                        epoch + 1)
        writer.flush()

        # Track best performance, and save the model's state
        if avg_vloss < best_vloss:
            best_vloss = avg_vloss
            torch.save(model.state_dict(), f"{model_path}/state_dict_epoch{epoch}.pt")

In [ ]:
# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.now().strftime('%Y%m%d_%Hh%Mm')
EPOCHS = 3

run_training(
    num_epochs=EPOCHS,
    model=model,
    train_loader=train_loader,
    validation_loader=val_loader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    timestamp=timestamp,
)

EPOCH 1:


100%|██████████| 247/247 [00:05<00:00, 43.82it/s]
/home/tim/Development/OCELFeatureExtractionExperiments/.env/lib/python3.9/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([786])) that is different to the input size (torch.Size([786, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/tim/Development/OCELFeatureExtractionExperiments/.env/lib/python3.9/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([832])) that is different to the input size (torch.Size([832, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/tim/Development/OCELFeatureExtractionExperiments/.env/lib/python3.9/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size

LOSS train 0 valid 0.9243588447570801
EPOCH 2:


100%|██████████| 247/247 [00:04<00:00, 50.91it/s]
/home/tim/Development/OCELFeatureExtractionExperiments/.env/lib/python3.9/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([773])) that is different to the input size (torch.Size([773, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/tim/Development/OCELFeatureExtractionExperiments/.env/lib/python3.9/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([861])) that is different to the input size (torch.Size([861, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/tim/Development/OCELFeatureExtractionExperiments/.env/lib/python3.9/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size

LOSS train 0 valid 0.9474873542785645
EPOCH 3:


...

LOSS train 0 valid 0.9494266510009766
EPOCH 29:


100%|██████████| 247/247 [00:05<00:00, 44.20it/s]


LOSS train 0 valid 0.9261217713356018
EPOCH 30:


100%|██████████| 247/247 [00:05<00:00, 45.54it/s]


LOSS train 0 valid 0.9289293885231018
